In [1]:
import keras as k
from keras.applications.vgg16 import VGG16
import pandas as pd
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import svm
from scipy.stats import mode

Using TensorFlow backend.


In [2]:
# Extract and create the data set
data = pd.read_csv("data/micrograph.csv")

In [3]:
data_spheroidite = data.loc[data['primary_microconstituent'] == 'spheroidite'].copy()
data_network = data.loc[data['primary_microconstituent'] == 'network'].copy()
data_pearlite = data.loc[data['primary_microconstituent'] == 'pearlite'].copy()


img = image.img_to_array(image.load_img("data/micrograph/"+data_spheroidite['path'].iloc[0]))
img = img[0:484,:,:]
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

In [4]:
model = VGG16(weights = 'imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=[model.get_layer(index=3).output, 
                                          model.get_layer(index=6).output,
                                          model.get_layer(index=10).output,
                                          model.get_layer(index=14).output,
                                          model.get_layer(index=18).output])



In [6]:
def extract_features (data):
    data['c12'] = [np.empty(1) for i in range(data.shape[0])]
    data['c22'] = [np.empty(1) for i in range(data.shape[0])]
    data['c33'] = [np.empty(1) for i in range(data.shape[0])]
    data['c43'] = [np.empty(1) for i in range(data.shape[0])]
    data['c53'] = [np.empty(1) for i in range(data.shape[0])]
    
    for index, row in data.iterrows():
        img = image.img_to_array(image.load_img("data/micrograph/"+row['path']))
        img = img[0:484,:,:]
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        c12, c22, c33, c43, c53 = intermediate_layer_model.predict(img)
        data.at[index,'c12'] = c12.mean(axis=(0,1,2,))
        data.at[index,'c22'] = c22.mean(axis=(0,1,2,))
        data.at[index,'c33'] = c33.mean(axis=(0,1,2,))
        data.at[index,'c43'] = c43.mean(axis=(0,1,2,))
        data.at[index,'c53'] = c53.mean(axis=(0,1,2,))

In [7]:
%timeit
extract_features (data_spheroidite)
%timeit
extract_features (data_network)
%timeit
extract_features (data_pearlite)

In [8]:
#train data

train_data_spher_netwk = pd.DataFrame ({'c12': np.append(data_spheroidite[0:100]['c12'].values, data_network[0:100]['c12'].values),
                                   'c22': np.append(data_spheroidite[0:100]['c22'].values, data_network[0:100]['c22'].values),
                                   'c33': np.append(data_spheroidite[0:100]['c33'].values, data_network[0:100]['c33'].values),
                                   'c43': np.append(data_spheroidite[0:100]['c43'].values, data_network[0:100]['c43'].values),
                                   'c53': np.append(data_spheroidite[0:100]['c53'].values, data_network[0:100]['c53'].values),
                                   'type': np.append(np.ones(100), 2*np.ones(100))}).sample(frac=1, random_state=0)

train_data_netwk_pearl = pd.DataFrame ({'c12': np.append(data_network[0:100]['c12'].values, data_pearlite[0:100]['c12'].values),
                                   'c22': np.append(data_network[0:100]['c22'].values, data_pearlite[0:100]['c22'].values),
                                   'c33': np.append(data_network[0:100]['c33'].values, data_pearlite[0:100]['c33'].values),
                                   'c43': np.append(data_network[0:100]['c43'].values, data_pearlite[0:100]['c43'].values),
                                   'c53': np.append(data_network[0:100]['c53'].values, data_pearlite[0:100]['c53'].values),
                                   'type': np.append(2*np.ones(100), 3*np.ones(100))}).sample(frac=1, random_state=0)

train_data_pearl_spher = pd.DataFrame ({'c12': np.append(data_pearlite[0:100]['c12'].values, data_spheroidite[0:100]['c12'].values),
                                   'c22': np.append(data_pearlite[0:100]['c22'].values, data_spheroidite[0:100]['c22'].values),
                                   'c33': np.append(data_pearlite[0:100]['c33'].values, data_spheroidite[0:100]['c33'].values),
                                   'c43': np.append(data_pearlite[0:100]['c43'].values, data_spheroidite[0:100]['c43'].values),
                                   'c53': np.append(data_pearlite[0:100]['c53'].values, data_spheroidite[0:100]['c53'].values),
                                   'type' : np.append(3*np.ones(100), np.ones(100))}).sample(frac=1, random_state=0)


In [9]:
# test_data

test_data_spher_netwk = pd.DataFrame ({'c12': np.append(data_spheroidite[100:]['c12'].values, data_network[100:]['c12'].values),
                                   'c22': np.append(data_spheroidite[100:]['c22'].values, data_network[100:]['c22'].values),
                                   'c33': np.append(data_spheroidite[100:]['c33'].values, data_network[100:]['c33'].values),
                                   'c43': np.append(data_spheroidite[100:]['c43'].values, data_network[100:]['c43'].values),
                                   'c53': np.append(data_spheroidite[100:]['c53'].values, data_network[100:]['c53'].values),
                                   'type' : np.append(np.ones(data_spheroidite[100:].shape[0]), 2*np.ones(data_network[100:].shape[0]))})

test_data_netwk_pearl = pd.DataFrame ({'c12': np.append(data_network[100:]['c12'].values, data_pearlite[100:]['c12'].values),
                                   'c22': np.append(data_network[100:]['c22'].values, data_pearlite[100:]['c22'].values),
                                   'c33': np.append(data_network[100:]['c33'].values, data_pearlite[100:]['c33'].values),
                                   'c43': np.append(data_network[100:]['c43'].values, data_pearlite[100:]['c43'].values),
                                   'c53': np.append(data_network[100:]['c53'].values, data_pearlite[100:]['c53'].values),
                                   'type' : np.append(2*np.ones(data_network[100:].shape[0]), 3*np.ones(data_pearlite[100:].shape[0]))})

test_data_pearl_spher = pd.DataFrame ({'c12': np.append(data_pearlite[100:]['c12'].values, data_spheroidite[100:]['c12'].values),
                                   'c22': np.append(data_pearlite[100:]['c22'].values, data_spheroidite[100:]['c22'].values),
                                   'c33': np.append(data_pearlite[100:]['c33'].values, data_spheroidite[100:]['c33'].values),
                                   'c43': np.append(data_pearlite[100:]['c43'].values, data_spheroidite[100:]['c43'].values),
                                   'c53': np.append(data_pearlite[100:]['c53'].values, data_spheroidite[100:]['c53'].values),
                                   'type' : np.append(3*np.ones(data_pearlite[100:].shape[0]), np.ones(data_spheroidite[100:].shape[0]))})


In [10]:
type(train_data_spher_netwk['type'].values)

numpy.ndarray

In [11]:
#spheroidite and network

# c12
sp_nt_clf12 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf12, list(train_data_spher_netwk['c12'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 error : ", 1.0 - scores['test_score'].mean())

# c22
sp_nt_clf22 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf22, list(train_data_spher_netwk['c22'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 error : ", 1.0 - scores['test_score'].mean())

# c33
sp_nt_clf33 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf33, list(train_data_spher_netwk['c33'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 error : ", 1.0 - scores['test_score'].mean())

# c43
sp_nt_clf43 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf43, list(train_data_spher_netwk['c43'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 error : ", 1.0 - scores['test_score'].mean())

# c53
sp_nt_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf53, list(train_data_spher_netwk['c53'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 error : ", 1.0 - scores['test_score'].mean())

c12 error :  0.29499999999999993
c22 error :  0.09499999999999997
c33 error :  0.025000000000000022
c43 error :  0.015000000000000013
c53 error :  0.010000000000000009


In [12]:
# c12 score :  0.7050000000000001
# c22 score :  0.905
# c33 score :  0.975
# c43 score :  0.985
# c53 score :  0.99

In [13]:
#network and pearlite

# c12
nt_pl_clf12 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf12, list(train_data_netwk_pearl['c12'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 error : ", 1.0 - scores['test_score'].mean())

# c22
nt_pl_clf22 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf22, list(train_data_netwk_pearl['c22'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 error : ", 1.0 - scores['test_score'].mean())

# c33
nt_pl_clf33 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf33, list(train_data_netwk_pearl['c33'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 error : ", 1.0 - scores['test_score'].mean())

# c43
nt_pl_clf43 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf43, list(train_data_netwk_pearl['c43'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 error : ", 1.0 - scores['test_score'].mean())

# c53
nt_pl_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf53, list(train_data_netwk_pearl['c53'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 error : ", 1.0 - scores['test_score'].mean())

c12 error :  0.11499999999999999
c22 error :  0.07499999999999996
c33 error :  0.03499999999999992
c43 error :  0.010000000000000009
c53 error :  0.0


In [14]:
#c12 score :  0.885
#c22 score :  0.925
#c33 score :  0.9650000000000001
#c43 score :  0.99
#c53 score :  1.0

In [15]:
#pearlite and spheroidite

# c12
pl_sp_clf12 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf12, list(train_data_pearl_spher['c12'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 error : ", 1.0 - scores['test_score'].mean())

# c22
pl_sp_clf22 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf22, list(train_data_pearl_spher['c22'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 error : ", 1.0 - scores['test_score'].mean())

# c33
pl_sp_clf33 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf33, list(train_data_pearl_spher['c33'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 error : ", 1.0 - scores['test_score'].mean())

# c43
pl_sp_clf43 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf43, list(train_data_pearl_spher['c43'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 error : ", 1.0 - scores['test_score'].mean())

# c53
pl_sp_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf53, list(train_data_pearl_spher['c53'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 error : ", 1.0 - scores['test_score'].mean())

c12 error :  0.1649999999999998
c22 error :  0.11499999999999999
c33 error :  0.019999999999999907
c43 error :  0.01000000000000012
c53 error :  0.010000000000000009


In [16]:
# c12 score :  0.8350000000000002
# c22 score :  0.885
# c33 score :  0.9800000000000001
# c43 score :  0.9899999999999999
# c53 score :  0.99

In [17]:
sp_nt_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
sp_nt_clf53.fit(list(train_data_spher_netwk['c53'].values), list(train_data_spher_netwk['type'].values))
print("SP_NT classifier test error : ", 1.0 - sp_nt_clf53.score(list(test_data_spher_netwk['c53'].values), list(test_data_spher_netwk['type'].values)))

SP_NT classifier test error :  0.010362694300518172


In [18]:
# 0.9896373056994818

In [19]:
nt_pl_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
nt_pl_clf53.fit(list(train_data_netwk_pearl['c53'].values), list(train_data_netwk_pearl['type'].values))
print("NT_PL classifier test error : ", 1.0 - nt_pl_clf53.score(list(test_data_netwk_pearl['c53'].values), list(test_data_netwk_pearl['type'].values)))

NT_PL classifier test error :  0.007352941176470562


In [20]:
# 0.9926470588235294

In [21]:
pl_sp_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
pl_sp_clf53.fit(list(train_data_pearl_spher['c53'].values), list(train_data_pearl_spher['type'].values))
print("PL_SP classifier test error  : ", 1.0 - pl_sp_clf53.score(list(test_data_pearl_spher['c53'].values), list(test_data_pearl_spher['type'].values)))

PL_SP classifier test error  :  0.010067114093959773


In [22]:
#0.9899328859060402

In [23]:
test_data = pd.DataFrame ({'c53': np.append(np.append(data_spheroidite[100:]['c53'].values, 
                            data_network[100:]['c53'].values),data_pearlite[100:]['c53']),
                           'type': np.append(np.append(np.ones(data_spheroidite[100:].shape[0]), 
                            2*np.ones(data_network[100:].shape[0])), 3*np.ones(data_pearlite[100:].shape[0]))})


In [31]:
pred_1 = sp_nt_clf53.predict(list(test_data['c53'].values))
pred_2 = nt_pl_clf53.predict(list(test_data['c53'].values))
pred_3 = pl_sp_clf53.predict(list(test_data['c53'].values))
pred = mode(np.concatenate((pred_1.reshape((pred_1.shape[0], 1)), pred_2.reshape((pred_2.shape[0], 1)),
                pred_3.reshape((pred_3.shape[0], 1)), np.zeros((pred_3.shape[0],1))), axis = 1), axis=1)[0]
truth_val = test_data['type'].values
truth_val = truth_val.reshape((truth_val.shape[0], 1))
print ("Total test error in 1 vs 1 : ", 1.0 - (truth_val[truth_val == pred].shape[0]/truth_val.shape[0]))

Total test error in 1 vs 1 :  0.017073170731707332


In [32]:
# For spherodite
pred_1 = sp_nt_clf53.predict(list(data_spheroidite[100:]['c53'].values))
pred_2 = pl_sp_clf53.predict(list(data_spheroidite[100:]['c53'].values))
correct_pred_sph = np.array(pred_1 == 1.0) & np.array(pred_2 == 1.0)

# For network
pred_1 = nt_pl_clf53.predict(list(data_network[100:]['c53'].values))
pred_2 = sp_nt_clf53.predict(list(data_network[100:]['c53'].values))
correct_pred_ntwk = np.array(pred_1 == 2.0) & np.array(pred_2 == 2.0)

# For pearlite
pred_1 = pl_sp_clf53.predict(list(data_pearlite[100:]['c53'].values))
pred_2 = nt_pl_clf53.predict(list(data_pearlite[100:]['c53'].values))
correct_pred_pl = np.array(pred_1 == 3.0) & np.array(pred_2 == 3.0)

In [33]:
prediction = np.append(np.append(correct_pred_sph, correct_pred_ntwk), correct_pred_pl)

total_performance = prediction[prediction == True].shape[0]/prediction.shape[0]
print ("Total error : ", 1.0 - total_performance)

Total error :  0.017073170731707332


In [27]:
data_spheroidite.to_csv('data/sperodite.csv', sep=',')
data_network.to_csv('data/network.csv', sep=',')
data_pearlite.to_csv('data/pearlite.csv', sep=',')

In [11]:
data_spheroidite.to_csv('data/sperodite_t.csv', sep='\t')
data_network.to_csv('data/network_t.csv', sep='\t')
data_pearlite.to_csv('data/pearlite_t.csv', sep='\t')

In [33]:
data_spheroidite['path'].iloc[0]

'micrograph2.tif'

In [37]:
?cross_validate

In [20]:
intermediate_output.shape

(1, 242, 322, 64)

In [52]:
a = [5, 6, 7, 8]
df = pd.DataFrame({"a": [5, 6 , 7 , 8], "b": [5, 6, 7, 8]})

In [62]:
df.sample(frac=1, random_state=0)

,a,b
2,7,7
3,8,8
1,6,6
0,5,5


In [39]:
df1 = pd.DataFrame({"a": np.append(y,y)})

In [40]:
df1

,a
0,5
1,6
2,5
3,6


In [117]:
def sam(df):
    for index, row in df.iterrows():
        print(index)
        row['a']=5

In [118]:
sam(df)

0
1


In [34]:
test_data_spher_netwk

,c12,c22,c33,c43,c53,type
0,"[256.0043, 126.03683, 141.11086, 89.13334, 400...","[248.23003, 441.21954, 317.6846, 868.9172, 285...","[313.53073, 417.88663, 372.75146, 60.16209, 25...","[16.287415, 3.541227, 51.515972, 33.59209, 40....","[0.16069059, 0.017117636, 0.6833866, 0.0157611...",1.0
1,"[395.86316, 194.411, 159.55864, 133.23584, 360...","[294.21298, 541.28204, 538.89594, 1059.4299, 4...","[371.42197, 593.4421, 242.23932, 58.75405, 564...","[6.913221, 13.379897, 3.7586145, 7.2396283, 40...","[0.026116168, 0.08166355, 0.19066846, 0.0, 0.4...",1.0
2,"[243.77591, 146.61656, 157.18938, 80.32311, 26...","[227.03966, 412.12924, 384.31277, 819.4552, 33...","[271.79803, 426.59534, 320.7263, 54.152447, 23...","[19.080511, 5.9287486, 60.45531, 58.40569, 36....","[0.011537608, 0.0060039535, 0.8913662, 0.05133...",1.0
3,"[183.42209, 206.87718, 130.55786, 88.475975, 3...","[291.40872, 584.0173, 437.82312, 739.2883, 882...","[490.6267, 525.6314, 621.3086, 438.2421, 553.2...","[9.081814, 16.346811, 34.616886, 32.98996, 172...","[1.4062133, 2.2620382, 5.285591, 3.2075715, 3....",1.0
4,"[307.69724, 198.6043, 155.3166, 113.758865, 29...","[313.77454, 595.21265, 551.8619, 846.2998, 688...","[667.28973, 321.4621, 424.35046, 391.15295, 61...","[15.572446, 14.188921, 29.315845, 88.69746, 11...","[0.42585278, 0.696847, 2.1358025, 0.0, 0.03774...",1.0
5,"[180.44583, 159.08035, 137.20842, 77.49959, 32...","[238.08052, 545.56036, 430.39313, 575.5057, 73...","[481.03647, 369.47195, 494.65463, 656.9612, 59...","[1.8624562, 13.462652, 28.615662, 26.518797, 1...","[3.4932256, 0.56522137, 4.1067243, 0.5694865, ...",1.0
6,"[96.428665, 61.68806, 121.07095, 25.495426, 38...","[79.12219, 120.41674, 97.82069, 129.37146, 66....","[82.00066, 241.23619, 90.479034, 157.64484, 70...","[2.8108976, 0.91892606, 4.741855, 11.643911, 2...","[0.5239092, 0.25145462, 0.49211675, 0.01044287...",1.0
7,"[483.9424, 245.84406, 160.11638, 163.76817, 39...","[303.10144, 812.11646, 807.1616, 939.3679, 579...","[857.6826, 593.489, 582.23883, 479.18417, 704....","[43.92639, 1.5771405, 23.059309, 29.071484, 70...","[0.926405, 1.0125268, 2.8504386, 0.0040095863,...",1.0
8,"[166.33487, 93.64811, 133.64731, 60.326843, 36...","[178.75108, 243.17686, 162.56221, 334.23172, 2...","[255.01779, 362.24316, 201.2607, 179.94844, 17...","[1.2568611, 18.087812, 21.607372, 67.78917, 10...","[0.32987657, 0.0, 3.8598092, 0.0, 0.6114217, 0...",1.0
9,"[259.70023, 144.70544, 152.02985, 90.355415, 3...","[232.51997, 514.30646, 453.41513, 706.8248, 37...","[477.01944, 314.25293, 425.95422, 259.3182, 43...","[29.714083, 9.995117, 52.576042, 54.7171, 67.0...","[5.0424404, 0.42914078, 5.351157, 0.47163066, ...",1.0


In [44]:
list(train_data_spher_netwk['c12'].values)[1]

array([341.3344  , 185.11885 , 150.824   , 112.44451 , 327.8188  ,
       275.09537 , 198.18904 , 102.41569 , 242.96877 , 136.56586 ,
       371.4226  , 115.533806, 225.17302 , 275.53955 ,  62.7535  ,
       116.48405 , 227.53041 , 272.0164  , 102.508705, 218.12967 ,
       336.26205 , 146.76839 , 143.13101 ,  22.458551, 185.24213 ,
       112.42335 , 245.81418 , 354.76944 , 238.9223  , 318.40817 ,
       276.80017 ,  75.56907 , 203.20589 , 224.17995 , 201.61795 ,
       207.20468 , 162.43324 , 297.76343 , 149.47534 ,  98.867065,
       274.83307 , 213.31392 , 256.21606 , 232.32637 , 216.25096 ,
        43.664513, 187.11922 , 392.18225 , 142.88338 ,  10.734444,
       340.461   , 115.634865, 317.44736 , 323.36084 ,   3.922971,
       326.60004 , 279.1217  , 253.99638 , 255.02556 , 299.32828 ,
        71.582535,  82.9605  , 293.3207  , 207.816   ], dtype=float32)

In [37]:
train_data_spher_netwk

NameError: name 'index' is not defined

In [113]:
a = np.array([1, 0, 0 , 1])
b = np.array([0, 1, 0 , 0])
np.array(a == 1) & np.array(b == 0)

array([ True, False, False,  True])

In [103]:
k = a == 1

In [93]:
l = b == 0

In [94]:
k

array([ True, False, False,  True])

In [95]:
l

array([ True, False,  True,  True])

In [105]:
k & l

array([ True, False, False,  True])

In [1]:
import numpy as np

In [4]:
 2*np.ones(5)

array([2., 2., 2., 2., 2.])

,c12,c22,c33,c43,c53,type
18,[6.94773616481753e-310],[6.94773616481674e-310],[6.9477361648183e-310],[6.94773616482306e-310],[6.9477361648183e-310],0.0
170,[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],1.0
107,[6.94773616481753e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.9477361648183e-310],[6.94773616481674e-310],1.0
98,[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],0.0
177,[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],1.0
182,[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],1.0
5,[0.0],[6.1201856e-316],[6.30752583e-316],[6.322581e-316],[6.3324702e-316],0.0
146,[6.9477361648183e-310],[6.94773616481674e-310],[6.9477361648183e-310],[6.94773616481674e-310],[6.94773616481674e-310],1.0
12,[6.94773616481753e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],0.0
152,[6.94773616481674e-310],[6.9477361648191e-310],[6.94773616481674e-310],[6.94773616481674e-310],[6.94773616481674e-310],1.0


In [13]:
list(train_data_spher_netwk['c12'].values)

[array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([0.]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([6.94773616e-310]),
 array([5.7652

In [35]:
X = np.array([[3,2,3], [1,2,2]])

from scipy.stats import mode

mode(X, axis=1)[0]

array([[3],
       [2]])

In [38]:
mode?

In [91]:
a = np.array ([1,1,1,1])
f = a
b = np.array([2,2,2,2])
c = np.array([3,3,3,3])
a = a.reshape((a.shape[0], 1))
b = b.reshape((b.shape[0], 1))
c = c.reshape((c.shape[0], 1))

In [92]:
d = np.concatenate((a,b,c,e), axis = 1)

In [87]:
d

array([[1., 2., 3., 0.],
       [1., 2., 3., 0.],
       [1., 2., 3., 0.],
       [1., 2., 3., 0.]])

In [90]:
e = np.zeros((4,1))

In [93]:
from scipy.stats import mode

mode(d, axis=1)[0]

array([[0.],
       [0.],
       [0.],
       [0.]])

In [98]:
a[(a==f)].shape

IndexError: boolean index did not match indexed array along dimension 1; dimension is 1 but corresponding boolean dimension is 4

In [99]:
truth_val = test_data['type'].values

In [100]:
truth_val

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [119]:
pred

array([[1., 3., 1., 0.],
       [1., 2., 1., 0.],
       [1., 3., 1., 0.],
       ...,
       [1., 3., 3., 0.],
       [1., 3., 3., 0.],
       [1., 3., 3., 0.]])

In [120]:
from scipy.stats import mode

mode(pred, axis=1)[0]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],